### Part 1: Extract Information from Dataset

**Data from:** Antezza, A., Frank, A., Frank, P., Franz, L., Rebinskaya, E. & Trebesch, C. (2022). "The Ukraine Support Tracker: Which countries help Ukraine and how?". Kiel Working Paper, No. 2218

In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.max_columns', 500)

In [3]:
original_df = pd.read_csv("../csv_files/aidEvents.csv", nrows = 783, usecols = list(range(0,21)))

In [4]:
original_df = original_df[["countries", "Announcement Date", "Type of Aid General", "Explanation", "Converted Value in EUR", "Source of Aid 1"]]

In [5]:
original_df["Converted Value in EUR"] = original_df["Converted Value in EUR"].replace(".", 0)

In [6]:
original_df["Converted Value in EUR"] = original_df["Converted Value in EUR"].apply( \
                                        lambda input:input.replace(",","") if type(input) == str else input)

In [7]:
original_df["Converted Value in EUR"] = original_df["Converted Value in EUR"].apply(lambda input: int(input) if type(input) == str else input)

In [8]:
original_df.columns = ["Country","Date","Type of Aid", "Description", "Monetary Value in EUR", "URL"]

In [9]:
original_df = original_df.dropna()

In [10]:
new_descriptions = []
for description in original_df["Description"]:
    res = description.split(".")[0] + "."
    if "Source of Aid 1" in res:
        res = res.replace("Source of Aid 1", "source")
    if "Source of Aid 2" in res:
        res = res.replace("Source of Aid 2", "source")
    new_descriptions.append(res)
    
original_df["new_descriptions"] = new_descriptions

In [11]:
import pycountry
three_letter_code = []
for country in original_df.Country:
    try:
        three_letter_code.append(pycountry.countries.search_fuzzy(country)[0].alpha_3)
    except:
        # print("Error retrieving code for: " + country)
        three_letter_code.append(None)
    
original_df["3-letter ISO code"] = three_letter_code

In [12]:
original_df["Monetary Value in EUR in str"] = [str(int(i)) for i in original_df["Monetary Value in EUR"]]

In [13]:
df = original_df.copy(deep=True)

In [14]:
df["text"] = "<b>" + df["Country"] + "</b> <br><br>" + "<b>Description</b> <br>" + df["Date"] + ":" \
                    + df["Description"]  + "(<b>" + df["Monetary Value in EUR in str"] + "</b>)" + "<br><br>" + "<b>Sources</b>"

In [15]:
paragraphs = {}

for country in set(df.Country):
    # print("Country is:" + country)
    paragraph = ""
    for row in df.itertuples():
        if row.Country == country:
            partial = "<b>" + row.Date + "</b>" + " " + row.new_descriptions + "<br>"
            paragraph += partial
            
    paragraphs[country] = paragraph
    
description_paragraphs = []
for row in df.itertuples():
    if row.Country in paragraphs:
        description_paragraphs.append(paragraphs[row.Country])
        
df["description_paragraph"] = description_paragraphs

In [16]:
sources = []
for row in df.itertuples():
    res = "<b>" + row.Date + "</b>: " + "<a href=\"" + row.URL + " target=\"_blank\" \">Source</a>"
    sources.append(res)
    
df["sources"] = sources

In [17]:
sources_paragraph = {}

for country in set(df.Country):
    # print("Country is:" + country)
    paragraph = ""
    for row in df.itertuples():
        if row.Country == country:
            partial = row.sources + "</br>"
            paragraph += partial
            
    sources_paragraph[country] = paragraph
    
sources_paragraph_list = []
for row in df.itertuples():
    if row.Country in sources_paragraph:
        sources_paragraph_list.append(sources_paragraph[row.Country])
        
df["sources_paragraph"] = sources_paragraph_list

In [18]:
df.columns

Index(['Country', 'Date', 'Type of Aid', 'Description',
       'Monetary Value in EUR', 'URL', 'new_descriptions', '3-letter ISO code',
       'Monetary Value in EUR in str', 'text', 'description_paragraph',
       'sources', 'sources_paragraph'],
      dtype='object')

In [19]:
df.to_csv("../csv_files/cleaned_aid_data.csv")

## Part Two: Plot Dataset using [Flourish](https://flourish.studio/)

In [20]:
import panel as pn
pn.extension()

def response_1():
    
    html_pane = pn.pane.HTML("""
    <iframe src='https://flo.uri.sh/visualisation/10587682/embed' title='Interactive or visual content' class='flourish-embed-iframe' frameborder='0' scrolling='no' style='width:100%;height:600px;' sandbox='allow-same-origin allow-forms allow-scripts allow-downloads allow-popups allow-popups-to-escape-sandbox allow-top-navigation-by-user-activation'></iframe><div style='width:100%!;margin-top:4px!important;text-align:right!important;'><a class='flourish-credit' href='https://public.flourish.studio/visualisation/10587682/?utm_source=embed&utm_campaign=visualisation/10587682' target='_top' style='text-decoration:none!important'><img alt='Made with Flourish' src='https://public.flourish.studio/resources/made_with_flourish.svg' style='width:105px!important;height:16px!important;border:none!important;margin:0!important;'> </a></div>
    """, style={'border': '2px solid black',
                'border-radius': '5px', 'padding': '10px', "height": "600px", "width": "1000px",})
    return html_pane

## Part 2: Plotting on a Geographical map using Plotly Express

In [21]:
# import plotly.express as px
# import plotly.graph_objects as go
# fig = go.Figure(go.Choropleth(
#                 locations = list(df.Country), 
#                 locationmode = "country names", 
#                 z = df["Monetary Value in EUR"],
#                 colorscale = px.colors.sequential.Cividis,
#                 colorbar_title="Monetary Aid Given to Ukraine",
#                 # hovertemplate = df["text"],
#                 hovertext=df["sources_paragraph"],
#                 zmin = 1000000,
#                 zmax = 8000000,
#                 # height =800,
#                   # width = 1000,
#                   # color_continuous_scale=px.colors.sequential.Cividis,
#                   # range_color=[1000000,20000000],
#                   # center = {"lat": 51.1657, "lon": 10.4515}, # Germany's coordinates
#                   # fitbounds = "locations",
#                 ))

# fig.update_layout(
#   geo_scope="europe",
#     height = 500,
#     width = 1000
# )

In [22]:
# import panel as pn
# pn.extension()

# def main_6():
    
#     html_pane = pn.pane.HTML("""
#     <iframe src='https://flo.uri.sh/story/1197131/embed#slide-0' title='Interactive or visual content' class='flourish-embed-iframe' frameborder='0' scrolling='no' style='width:100%;height:600px;' sandbox='allow-same-origin allow-forms allow-scripts allow-downloads allow-popups allow-popups-to-escape-sandbox allow-top-navigation-by-user-activation'></iframe><div style='width:100%!;margin-top:4px!important;text-align:right!important;'><a class='flourish-credit' href='https://public.flourish.studio/visualisation/10586999/?utm_source=embed&utm_campaign=visualisation/10586999' target='_top' style='text-decoration:none!important'><img alt='Made with Flourish' src='https://public.flourish.studio/resources/made_with_flourish.svg' style='width:105px!important;height:16px!important;border:none!important;margin:0!important;'> </a></div>
#     """, style={'border': '2px solid black',
#                 'border-radius': '5px', 'padding': '10px', "height": "600px", "width": "1000px",})
#     return html_pane

In [23]:
# data = dict(type = "choropleth", 
#             locations = list(original_df.countries), 
#             locationmode = "country names", 
#             z = "Converted Value in EUR", 
#             )

# fig = px.choropleth(original_df, 
#                   locations="countries",
#                   locationmode = "country names",
#                   color = "Converted Value in EUR",
#                   scope="europe",
#                   height =800,
#                   width = 1000,
#                   color_continuous_scale=px.colors.sequential.Cividis,
#                   range_color=[1000000,20000000],
#                   center = {"lat": 51.1657, "lon": 10.4515}, # Germany's coordinates
#                   fitbounds = "locations",
#                   text=df['Announcement Date'],
#                   )

# fig.show()

In [24]:
# import plotly.graph_objs as go
# from ipywidgets import Output, VBox

# fig = go.FigureWidget()
# pie = fig.add_pie(values=[1, 2, 3])

# out = Output()
# @out.capture(clear_output=True)
# def handle_click(trace, points, state):
#     print(points.point_inds)

# pie.on_click(handle_click)

# VBox([fig, out])

In [25]:
# import plotly.graph_objects as go
# import webbrowser

# from ipywidgets import Output, VBox

# import numpy as np
# np.random.seed(1)

# x = np.random.rand(100)
# y = np.random.rand(100)

# f = go.FigureWidget([go.Scatter(x=x, y=y, mode="markers")])

# scatter = f.data[0]
# colors = ["#a3a7e4"] * 100
# scatter.marker.color = colors
# scatter.marker.size = [10] * 100
# f.layout.hovermode = "closest"

# out = Output()
# @out.capture(clear_output=True)
# def update_point(trace, points, selector):
#     c = list(scatter.marker.color)
#     s = list(scatter.marker.size)
#     for i in points.point_inds:
#         c[i] = "#bae2be"
#         s[i] = 20
#     with f.batch_update():
#         scatter.marker.color = c
#         scatter.marker.size = s
#         webbrowser.open("http://example.com 28")

# scatter.on_click(update_point)
# VBox([f, out])